In [11]:
import math
import numpy as np
import pandas as pd

df = pd.read_feather("firma_data_better.feather")

In [12]:
print("Columns:", df.columns.tolist())
print("Date range:", df['Date'].iloc[0], "to", df['Date'].iloc[-1])
#df['Date'] = pd.to_datetime(df['Date'])

Columns: ['Date', 'Underlying', 'C445', 'C450', 'C455', 'C460', 'C465', 'C470', 'C475', 'C480', 'C485', 'C490', 'C495', 'C500', 'C505', 'C510', 'C515', 'C520', 'P445', 'P450', 'P455', 'P460', 'P465', 'P470', 'P475', 'P480', 'P485', 'P490', 'P495', 'P500', 'P505', 'P510', 'P515', 'P520']
Date range: 2025-09-11 00:00:00 to 2025-10-31 00:00:00


In [13]:
# Choose a strike for the ATM call (here 500, which has no missing prices in the series)
strike = 470
option_col = f"C{strike}"

In [14]:
# Check if the chosen option has any missing values
missing_count = df[option_col].isna().sum()
print(f"Missing data points for strike {strike} call:", missing_count)
# If there are missing prices, we can interpolate or drop those days. Interpolation is chosen here.
if missing_count > 0:
    df[option_col] = df[option_col].interpolate()  # simple linear interpolation for any minor gaps

Missing data points for strike 470 call: 12


In [15]:
# Extract relevant series for underlying and option
S_series = df['Underlying'].values
C_series = df[option_col].values
N = len(C_series)  # number of days in the series
print(f"Using strike {strike} call, total days = {N}")
print(df[['Date', 'Underlying', option_col]].head())
print(df[['Date', 'Underlying', option_col]].tail())

Using strike 470 call, total days = 37
        Date  Underlying   C470
0 2025-09-11      470.73   18.2
1 2025-09-12      471.31  18.55
2 2025-09-15      473.25   18.9
3 2025-09-16      474.32  21.39
4 2025-09-17      473.12  19.35
         Date  Underlying   C470
32 2025-10-27      486.91   17.1
33 2025-10-28      485.77  15.53
34 2025-10-29      485.33   14.6
35 2025-10-30      489.72  23.35
36 2025-10-31      491.88  19.04


In [16]:

# Cumulative distribution function for standard normal
def norm_cdf(x: float) -> float:
    return 0.5 * (1.0 + math.erf(x / math.sqrt(2.0)))

# Black-Scholes formula for European call price (no dividends assumed)
def black_scholes_call_price(S: float, K: float, T: float, r: float, sigma: float) -> float:
    """Compute Black-Scholes price of a European call option."""
    if T <= 1e-16:  # effectively at maturity
        return max(0.0, S - K)
    if sigma <= 1e-8:  # near zero volatility
        # If volatility is zero, option value is just discounted intrinsic value (if any)
        return max(0.0, S - K * math.exp(-r * T))
    # Black-Scholes formula components
    d1 = (math.log(S/K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    # Call price = S * N(d1) - K * e^{-rT} * N(d2)
    return S * norm_cdf(d1) - K * math.exp(-r * T) * norm_cdf(d2)

# Black-Scholes Delta for a call option
def black_scholes_delta(S: float, K: float, T: float, r: float, sigma: float) -> float:
    """Compute Black-Scholes delta of a call option."""
    if T <= 1e-16:
        # At maturity, delta is 1 if option finishes in the money, else 0
        return 1.0 if S > K else 0.0
    if sigma <= 1e-8:
        # If essentially zero volatility, delta is 1 if forward S would exceed K, else 0
        return 1.0 if S * math.exp(r * T) > K else 0.0
    d1 = (math.log(S/K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    return norm_cdf(d1)

# Black-Scholes Vega for a call option (sensitivity to volatility)
def black_scholes_vega(S: float, K: float, T: float, r: float, sigma: float) -> float:
    """Compute Black-Scholes vega of a call option."""
    if T <= 1e-16 or sigma <= 1e-8:
        return 0.0
    d1 = (math.log(S/K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    # Standard normal probability density
    pdf = (1.0 / math.sqrt(2 * math.pi)) * math.exp(-0.5 * d1**2)
    # Vega = S * sqrt(T) * pdf (no dividend case)
    return S * math.sqrt(T) * pdf

# Implied volatility solver using bisection
def implied_volatility_call(C_market: float, S: float, K: float, T: float, r: float=0.0) -> float:
    """Find implied volatility for a call given its market price using bisection."""
    # Handle trivial cases at expiration:
    if T <= 1e-16:
        # If at maturity, implied vol is 0 if price equals intrinsic value
        intrinsic = max(0.0, S - K)
        return 0.0 if abs(C_market - intrinsic) < 1e-8 else float('nan')
    # Initialize bounds for vol
    low_vol, high_vol = 0.0, 5.0  # search between 0% and 500% vol
    # Ensure target price is within bounds of model prices:
    price_low = black_scholes_call_price(S, K, T, r, low_vol + 1e-8)  # near-zero vol price
    price_high = black_scholes_call_price(S, K, T, r, high_vol)
    if C_market < price_low - 1e-8 or C_market > price_high + 1e-8:
        # Price out of feasible range (could happen due to arbitrage or data error)
        return float('nan')
    # Bisection iteration
    for _ in range(100):
        mid_vol = 0.5 * (low_vol + high_vol)
        price_mid = black_scholes_call_price(S, K, T, r, mid_vol)
        if abs(price_mid - C_market) < 1e-6:
            return mid_vol
        if price_mid < C_market:
            low_vol = mid_vol
        else:
            high_vol = mid_vol
    # Return mid of bounds if no exact convergence
    return 0.5 * (low_vol + high_vol)

# Quick test of the functions on the first day of data
r = 0.0  # assume zero risk-free rate for simplicity
S0 = S_series[0]
C0 = C_series[0]
T0 = (N - 1) / 252.0  # initial time to maturity in years (~45 trading days)
imp_vol0 = implied_volatility_call(C0, S0, strike, T0, r)
delta0 = black_scholes_delta(S0, strike, T0, r, imp_vol0)
vega0 = black_scholes_vega(S0, strike, T0, r, imp_vol0)
print(f"Initial implied vol ≈ {imp_vol0:.2%}, Delta ≈ {delta0:.3f}, Vega ≈ {vega0:.2f}")

Initial implied vol ≈ 25.15%, Delta ≈ 0.525, Vega ≈ 70.83


In [17]:
import numpy as np

def simulate_delta_hedge(S: np.ndarray, C: np.ndarray, K: float, r: float=0.0, rebalance_freq: int=1):
    """Simulate delta hedging for the option price series C with underlying S.
    rebalance_freq = 1 for daily, 2 for every 2 days, etc."""
    n_days = len(C)
    errors = []  # store A_i errors
    current_delta = None
    for i in range(n_days - 1):
        # Rebalance on day i if it's a rebalance point
        if i % rebalance_freq == 0:
            T = (n_days - 1 - i) / 252.0  # remaining time in years
            vol = implied_volatility_call(C[i], S[i], K, T, r)
            if math.isnan(vol):
                # Skip if implied vol cannot be computed (should not happen if data chosen properly)
                vol = 0.0
            current_delta = black_scholes_delta(S[i], K, T, r, vol)
        # Compute error A_i for day i->i+1 using current_delta
        dS = S[i+1] - S[i]
        dC = C[i+1] - C[i]
        A_i = dC - current_delta * dS
        errors.append(A_i)
    errors = np.array(errors)
    mse = np.mean(errors**2)
    return errors, mse

# Run delta hedging simulation for different rebalancing frequencies
freqs = [1, 2, 5]  # daily, every 2 days, weekly
results_delta = {}
for freq in freqs:
    errs, mse = simulate_delta_hedge(S_series, C_series, strike, r=0.0, rebalance_freq=freq)
    results_delta[freq] = mse
    print(f"Delta hedge MSE with rebalancing every {freq} day(s): {mse:.6f}")

Delta hedge MSE with rebalancing every 1 day(s): 16.923516
Delta hedge MSE with rebalancing every 2 day(s): 14.267012
Delta hedge MSE with rebalancing every 5 day(s): 13.025846


In [18]:
def simulate_delta_vega_hedge(S: np.ndarray, C: np.ndarray, K: float, 
                              r: float=0.0, rebalance_freq: int=1, T_factor: float=2.0):
    """Simulate delta-vega hedging. 
    T_factor: how much longer the replicating option's maturity is relative to original (e.g. 2.0 = double)."""
    n_days = len(C)
    # Determine the replicating option's initial remaining days (approximately T_factor times original)
    rep_total_days = int((n_days - 1) * T_factor)  
    errors = []
    current_delta = current_n = 0.0
    for i in range(n_days - 1):
        # Compute time to maturity for original and replicating options on day i
        T1 = (n_days - 1 - i) / 252.0             # time remaining for original option
        T2 = max((rep_total_days - i) / 252.0, 0)  # time remaining for replicating option
        # Rebalance positions at specified frequency
        if i % rebalance_freq == 0:
            # Compute implied vol from original option price
            vol = implied_volatility_call(C[i], S[i], K, T1, r)
            if math.isnan(vol):
                vol = 0.0
            # Compute greeks for original option
            delta_orig = black_scholes_delta(S[i], K, T1, r, vol)
            vega_orig = black_scholes_vega(S[i], K, T1, r, vol)
            # Compute greeks for replicating option (using same vol assumption)
            delta_rep = black_scholes_delta(S[i], K, T2, r, vol)
            vega_rep = black_scholes_vega(S[i], K, T2, r, vol)
            # Determine replicating option amount to short (n) to neutralize vega
            if vega_rep < 1e-8:
                # Avoid division by zero if vega is extremely small
                current_n = 0.0
            else:
                current_n = vega_orig / vega_rep
            # Determine stock delta to short to neutralize net delta
            current_delta = delta_orig - current_n * delta_rep
        # Compute daily changes
        dS = S[i+1] - S[i]
        dC_orig = C[i+1] - C[i]
        # Simulate replicating call price change using Black-Scholes model
        # (We update its price based on underlying change and any implied vol change day-to-day)
        T1_next = (n_days - 1 - (i+1)) / 252.0
        T2_next = max((rep_total_days - (i+1)) / 252.0, 0)
        # For simplicity, assume implied vol on day i+1 equals that of day i (or recalc from C[i+1])
        vol_next = implied_volatility_call(C[i+1], S[i+1], K, T1_next, r)
        if math.isnan(vol_next):
            vol_next = 0.0
        # Price of replicating call on day i and i+1
        C_rep_i = black_scholes_call_price(S[i], K, T2, r, vol if not math.isnan(vol) else 0.0)
        C_rep_next = black_scholes_call_price(S[i+1], K, T2_next, r, vol_next)
        dC_rep = C_rep_next - C_rep_i
        # Hedging error for day i
        A_i = dC_orig - current_delta * dS - current_n * dC_rep
        errors.append(A_i)
    errors = np.array(errors)
    mse = np.mean(errors**2)
    return errors, mse

# Run delta-vega hedging simulation for different rebalancing frequencies (daily, 2-day, weekly)
results_deltavega = {}
for freq in freqs:
    errs, mse = simulate_delta_vega_hedge(S_series, C_series, strike, r=0.0, rebalance_freq=freq, T_factor=2.0)
    results_deltavega[freq] = mse
    print(f"Delta-Vega hedge MSE with rebalancing every {freq} day(s): {mse:.6f}")


Delta-Vega hedge MSE with rebalancing every 1 day(s): 7.626584
Delta-Vega hedge MSE with rebalancing every 2 day(s): 9.568200
Delta-Vega hedge MSE with rebalancing every 5 day(s): 11.701399


In [19]:
import pandas as pd
import QuantLib as ql
import numpy as np
from datetime import datetime

# Load the data (adjust path as needed)
df = pd.read_csv("firma_data_better.csv")
df['Date'] = pd.to_datetime(df['Date'])

# Identify the ATM call strike
start_price = df['Underlying'].iloc[0]
call_strikes = [int(col[1:]) for col in df.columns if col.startswith('C')]
atm_strike = 500#min(call_strikes, key=lambda k: abs(k - start_price))
option_col = f'C{atm_strike}'

# Clean the dataset
df = df[['Date', 'Underlying', option_col]].dropna()
df.columns = ['Date', 'Underlying', 'CallPrice']
df.reset_index(drop=True, inplace=True)

# QuantLib setup
calendar = ql.UnitedStates(ql.UnitedStates.NYSE)
day_counter = ql.ActualActual(ql.ActualActual.ISDA)
eval_date = ql.Date(df['Date'].iloc[0].day, df['Date'].iloc[0].month, df['Date'].iloc[0].year)
ql.Settings.instance().evaluationDate = eval_date

# Hedging parameters
spot_handle = ql.QuoteHandle(ql.SimpleQuote(df['Underlying'].iloc[0]))
flat_ts = ql.YieldTermStructureHandle(ql.FlatForward(eval_date, 0.0, day_counter))
flat_div = ql.YieldTermStructureHandle(ql.FlatForward(eval_date, 0.0, day_counter))

# Containers
deltas, vegas, ivs, errors = [], [], [], []

# Loop over each day to compute Greeks and implied vol
for i in range(len(df) - 1):
    today = df['Date'].iloc[i]
    maturity_date = ql.Date(df['Date'].iloc[-1].day, df['Date'].iloc[-1].month, df['Date'].iloc[-1].year)
    T = day_counter.yearFraction(ql.Date(today.day, today.month, today.year), maturity_date)

    S = df['Underlying'].iloc[i]
    price = df['CallPrice'].iloc[i]
    strike = atm_strike

    spot_handle = ql.QuoteHandle(ql.SimpleQuote(S))

    # Initial guess for volatility
    vol_guess = 0.2
    vol_handle = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(eval_date, calendar, vol_guess, day_counter))

    process = ql.BlackScholesMertonProcess(spot_handle, flat_div, flat_ts, vol_handle)
    payoff = ql.PlainVanillaPayoff(ql.Option.Call, strike)
    exercise = ql.EuropeanExercise(maturity_date)
    option = ql.VanillaOption(payoff, exercise)
    option.setPricingEngine(ql.AnalyticEuropeanEngine(process))

    try:
        implied_vol = option.impliedVolatility(price, process)
    except RuntimeError:
        implied_vol = np.nan

    if np.isnan(implied_vol):
        deltas.append(np.nan)
        vegas.append(np.nan)
        ivs.append(np.nan)
        errors.append(np.nan)
        continue

    vol_handle = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(eval_date, calendar, implied_vol, day_counter))
    process = ql.BlackScholesMertonProcess(spot_handle, flat_div, flat_ts, vol_handle)
    option.setPricingEngine(ql.AnalyticEuropeanEngine(process))

    greeks = {}
    greeks['delta'] = option.delta()
    greeks['vega'] = option.vega()

    dS = df['Underlying'].iloc[i+1] - df['Underlying'].iloc[i]
    dC = df['CallPrice'].iloc[i+1] - df['CallPrice'].iloc[i]
    hedge_error = dC - greeks['delta'] * dS

    deltas.append(greeks['delta'])
    vegas.append(greeks['vega'])
    ivs.append(implied_vol)
    errors.append(hedge_error)

# Compute mean squared error
mse = np.nanmean(np.array(errors) ** 2)

print(f"ATM Strike: {atm_strike}")
print(f"Final Delta Hedge MSE: {mse:.6f}")


ATM Strike: 500
Final Delta Hedge MSE: 0.980308
